In [ ]:
import tkinter as tk
from tkinter import ttk, scrolledtext
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder

# Load the pre-trained XGBoost model
with open('xgb_model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)
    
# Load MinMaxScaler
with open('min_max_scaler.pkl', 'rb') as scaler_file:
    scaler = pickle.load(scaler_file)

# Load LabelEncoder for 'brand'
with open('label_encoder_brand.pkl', 'rb') as label_encoder_file:
    label_encoder_brand = pickle.load(label_encoder_file)

# Function to perform feature engineering
def perform_feature_engineering(df):
    # Drop unnecessary columns
    df = df.drop(['seller', 'offerType', 'model', 'name', 'dateCrawled', 'lastSeen', 'dateCreated', 'postalCode'], axis=1)

    # Creating dummies
    df = pd.get_dummies(df, columns=['abtest','vehicleType', 'fuelType', 'gearbox', 'notRepairedDamage'], drop_first=True)

    # Label encoding for the 'brand' column
    label_encoder = LabelEncoder()
    df['brand_encoded'] = label_encoder.fit_transform(df['brand'])

    # Dropping brand column
    df = df.drop('brand', axis=1)
    
    # Convert 'yearOfRegistration' to numeric
    df['yearOfRegistration'] = pd.to_numeric(df['yearOfRegistration'], errors='coerce')

    # Calculate the age of the vehicle
    df['yearOfRegistration'] = df['yearOfRegistration'].apply(lambda x: 2023 - x)

    # Rename column name from 'yearOfRegistration' to 'YearsFromRegistration'
    df.rename(columns={'YearsFromRegistration': 'yearOfRegistration'}, inplace=True)

    # Convert 'yearOfRegistration' to numeric
    df['monthOfRegistration'] = pd.to_numeric(df['monthOfRegistration'], errors='coerce')
    
    # Convert 'yearOfRegistration' to numeric
    df['powerPS'] = pd.to_numeric(df['powerPS'], errors='coerce')
    
    # Convert 'yearOfRegistration' to numeric
    df['kilometer'] = pd.to_numeric(df['kilometer'], errors='coerce')
    
    # Perform binning for "monthOfRegistration" column
    bins = [0, 3, 6, 9, 12]
    labels = ["Q1", "Q2", "Q3", "Q4"]
    df['monthOfRegistration'] = pd.cut(df['monthOfRegistration'], bins=bins, labels=labels)
    df = pd.get_dummies(df, columns=['monthOfRegistration'], drop_first=True)

    return df

# Function to predict price
def predict_price(features):
    # Load the features into a DataFrame
    input_data = pd.DataFrame(features, index=[0])

    # Perform feature engineering
    input_data = perform_feature_engineering(input_data)

    # Predict the price using the pre-trained model
    predicted_price = model.predict(input_data)

    return predicted_price[0]

# Tkinter GUI
class CarPricePredictionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Car Price Prediction App")

        # Create a scrolled text widget for displaying the output
        self.output_text = scrolledtext.ScrolledText(root, width=50, height=10)
        self.output_text.grid(row=0, column=2, rowspan=20, padx=10, pady=10, sticky="nsew")

        # Create and place input labels and entry widgets
        self.input_entries = {}
        input_features = ['dateCrawled', 'name', 'seller', 'offerType', 'abtest', 'vehicleType', 'yearOfRegistration',
                           'gearbox', 'powerPS', 'model', 'kilometer', 'monthOfRegistration', 'fuelType', 'brand',
                           'notRepairedDamage', 'dateCreated', 'postalCode', 'lastSeen']

        for feature in input_features:
            label = ttk.Label(root, text=feature)
            label.grid(row=input_features.index(feature), column=0, padx=10, pady=10, sticky=tk.W)

            entry = ttk.Entry(root)
            entry.grid(row=input_features.index(feature), column=1, padx=10, pady=10, sticky=tk.W)
            self.input_entries[feature] = entry

        # Create and place the Predict button
        predict_button = ttk.Button(root, text="Predict Price", command=self.predict_price)
        predict_button.grid(row=len(input_features), column=0, columnspan=2, pady=20)

    def predict_price(self):
        # Get user inputs
        features = {feature: entry.get() for feature, entry in self.input_entries.items()}

        # Perform prediction
        predicted_price = predict_price(features)

        # Display the predicted price
        self.output_text.delete(1.0, tk.END)  # Clear previous text
        self.output_text.insert(tk.END, f"Predicted Price: {predicted_price:.2f} USD\n")

if __name__ == "__main__":
    root = tk.Tk()
    app = CarPricePredictionApp(root)

    # Configure the grid to expand with the window
    root.columnconfigure(2, weight=1)
    root.rowconfigure(0, weight=1)

    root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Jagadeeshilpi\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\JAGADE~1\AppData\Local\Temp/ipykernel_8636/3660736509.py", line 98, in predict_price
    predicted_price = predict_price(features)
  File "C:\Users\JAGADE~1\AppData\Local\Temp/ipykernel_8636/3660736509.py", line 61, in predict_price
    predicted_price = model.predict(input_data)
  File "C:\Users\Jagadeeshilpi\anaconda3\lib\site-packages\xgboost\sklearn.py", line 1164, in predict
    predts = self.get_booster().inplace_predict(
  File "C:\Users\Jagadeeshilpi\anaconda3\lib\site-packages\xgboost\sklearn.py", line 738, in get_booster
    raise NotFittedError("need to call fit or load_model beforehand")
sklearn.exceptions.NotFittedError: need to call fit or load_model beforehand


In [ ]:
import tkinter as tk
from tkinter import ttk, scrolledtext
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder

# Load the pre-trained XGBoost model
with open('xgb_model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

# Function to perform feature engineering
def perform_feature_engineering(df):
    # Drop unnecessary columns
    df = df.drop(['seller', 'offerType', 'model', 'name', 'dateCrawled', 'lastSeen', 'dateCreated', 'postalCode'], axis=1)

    # Creating dummies
    df = pd.get_dummies(df, columns=['abtest','vehicleType', 'fuelType', 'gearbox', 'notRepairedDamage'], drop_first=True)

    # Label encoding for the 'brand' column
    label_encoder = LabelEncoder()
    df['brand_encoded'] = label_encoder.fit_transform(df['brand'])

    # Dropping brand column
    df = df.drop('brand', axis=1)
    
    # Convert 'yearOfRegistration' to numeric
    df['yearOfRegistration'] = pd.to_numeric(df['yearOfRegistration'], errors='coerce')

    # Calculate the age of the vehicle
    df['yearOfRegistration'] = df['yearOfRegistration'].apply(lambda x: 2023 - x)

    # Rename column name from 'yearOfRegistration' to 'YearsFromRegistration'
    df.rename(columns={'YearsFromRegistration': 'yearOfRegistration'}, inplace=True)

    # Convert 'yearOfRegistration' to numeric
    df['monthOfRegistration'] = pd.to_numeric(df['monthOfRegistration'], errors='coerce')
    
    # Convert 'yearOfRegistration' to numeric
    df['powerPS'] = pd.to_numeric(df['powerPS'], errors='coerce')
    
    # Convert 'yearOfRegistration' to numeric
    df['kilometer'] = pd.to_numeric(df['kilometer'], errors='coerce')
    
    # Perform binning for "monthOfRegistration" column
    bins = [0, 3, 6, 9, 12]
    labels = ["Q1", "Q2", "Q3", "Q4"]
    df['monthOfRegistration'] = pd.cut(df['monthOfRegistration'], bins=bins, labels=labels)
    df = pd.get_dummies(df, columns=['monthOfRegistration'], drop_first=True)

    return df

# Function to predict price
def predict_price(features):
    # Load the features into a DataFrame
    input_data = pd.DataFrame(features, index=[0])

    # Perform feature engineering
    input_data = perform_feature_engineering(input_data)

    # Predict the price using the pre-trained model
    predicted_price = model.predict(input_data)

    return predicted_price[0]

# Tkinter GUI
class CarPricePredictionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Car Price Prediction App")

        # Create a scrolled text widget for displaying the output
        self.output_text = scrolledtext.ScrolledText(root, width=50, height=10)
        self.output_text.grid(row=0, column=2, rowspan=20, padx=10, pady=10, sticky="nsew")

        # Create and place input labels and entry widgets
        self.input_entries = {}
        input_features = ['dateCrawled', 'name', 'seller', 'offerType', 'abtest', 'vehicleType', 'yearOfRegistration',
                           'gearbox', 'powerPS', 'model', 'kilometer', 'monthOfRegistration', 'fuelType', 'brand',
                           'notRepairedDamage', 'dateCreated', 'postalCode', 'lastSeen']

        for feature in input_features:
            label = ttk.Label(root, text=feature)
            label.grid(row=input_features.index(feature), column=0, padx=10, pady=10, sticky=tk.W)

            entry = ttk.Entry(root)
            entry.grid(row=input_features.index(feature), column=1, padx=10, pady=10, sticky=tk.W)
            self.input_entries[feature] = entry

        # Create and place the Predict button
        predict_button = ttk.Button(root, text="Predict Price", command=self.predict_price)
        predict_button.grid(row=len(input_features), column=0, columnspan=2, pady=20)

    def predict_price(self):
        # Get user inputs
        features = {feature: entry.get() for feature, entry in self.input_entries.items()}

        # Perform prediction
        predicted_price = predict_price(features)

        # Display the predicted price
        self.output_text.delete(1.0, tk.END)  # Clear previous text
        self.output_text.insert(tk.END, f"Predicted Price: {predicted_price:.2f} USD\n")

if __name__ == "__main__":
    root = tk.Tk()
    app = CarPricePredictionApp(root)

    # Configure the grid to expand with the window
    root.columnconfigure(2, weight=1)
    root.rowconfigure(0, weight=1)

    root.mainloop()